In [ ]:
%load_ext autoreload
%autoreload 2

https://colab.research.google.com/drive/1uCuA6We9T5r0WljspEHWPHXCT_2bMKUy

In [ ]:
from climbing_wire.homography.homography import compute_homography
from climbing_wire.homography.homography import perspective_transform
from climbing_wire.landmark.compute import PoseImg
from climbing_wire.landmark.compute import compute_landmarks
from climbing_wire.landmark.drawing import draw_landmarks
from climbing_wire.landmark.landmark_list import LandmarkListImg
from climbing_wire.utils.cv import cv_imshow
from climbing_wire.utils.data import get_package_fol
from climbing_wire.utils.mediapipe import JOINT_NAMES
from climbing_wire.utils.mediapipe import get_default_pose_connections
from climbing_wire.video.load import iterate_video_frames, pairwise_video_frames
from matplotlib import pyplot as plt
from pathlib import Path
from typing import List, Tuple, cast
import cv2 as cv
import math
import matplotlib.pyplot as plt
import mediapipe as mp
import mediapipe.python.solutions.drawing_styles as mp_drawing_styles
import mediapipe.python.solutions.drawing_utils as mp_drawing
import mediapipe.python.solutions.pose as mp_pose
import numpy as np
import pathlib
import sys


In [ ]:
pose_connections = get_default_pose_connections()


In [ ]:
img_fol = get_package_fol("sample_square")


In [ ]:
# Read images with OpenCV.
imgs = {img_path.name: cv.imread(str(img_path)) for img_path in img_fol.iterdir()}
img_names = list(imgs)

img1 = imgs[img_names[0]]


In [ ]:
# test NM's video

# in_fol = Path("/mnt/c/Users/nobilip/Videos/dataset/rock")
# in_fn = "PXL_20220901_202119640.mp4"

in_fol = Path("~/data/rock").expanduser()
in_fn = "nonaka_miho_01.mp4"

in_vid_path = in_fol / in_fn

_, img1 = next(pairwise_video_frames(in_vid_path, msec_interval=10000))

# test if in a smaller image the person is detected
pad = 250
img1 = img1[
    400 - pad : 1300 + pad,
    300 - pad : 900 + pad,
    :,
]
print(f"{img1.shape=}")
cv_imshow(img1)


A manazza compute the landmarks using a Pose object

In [ ]:
from climbing_wire.landmark.landmark_list import LandmarkListNp


pose = mp_pose.Pose(
    static_image_mode=True,
    min_detection_confidence=0.5,
    model_complexity=2,
)

pose_landmarks = compute_landmarks(img1, pose)

if pose_landmarks is not None:

    # llnp = LandmarkListNp(pose_landmarks)
    # print(f"{llnp!r}")

    annotated_image = img1.copy()
    mp_drawing.draw_landmarks(
        annotated_image,
        pose_landmarks,
        pose_connections,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
    )
    cv_imshow(annotated_image)


Use the `PoseImg` wrapper and drawing utils.

In [ ]:
pl = PoseImg(
    static_image_mode=True,
    min_detection_confidence=0.5,
    model_complexity=2,
)
print(f"{pl=}")

pose_landmarks = pl(img1)
# print(f"{pose_landmarks!r}")

if pose_landmarks is not None:
    annotated_image = img1.copy()
    print(f"{annotated_image.shape=}")
    draw_landmarks(
        annotated_image,
        pose_landmarks,
        pose_connections,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
    )
    cv_imshow(annotated_image)
